<a href="https://colab.research.google.com/github/wantor-stack/Coursera_Capstone/blob/main/Clustering_Rio_de_Janeiro_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Applied Data Science Capstone- Battle of the Neighborhoods
## Clustering the Neighborhoods of Rio de Janeiro, RJ

## Author: Wanderson Torres


---



---


## Introduction/ Business Problem:

#### The emergence of the coronavirus disease 2019 (COVID-19), which is caused by infection from the previously unknown severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), has devastated economies and caused unprecedented challenges to healthcare and food systems around the world. Globally, billions of people have been ordered to stay at home as a result of lockdowns, while almost three million people have died (as of the end of March 2021).

#### With the pandemic infecting the planet, most jobs have gone remote, thus giving people an option to move farther away from the office to a safety area. We have seen a trend where people are moving away from crowded and expensive neighborhoods to the suburbs or even to other cities/states where there is more secure and more protected against the virus. 

#### I live in the city of Rio de Janeiro and due to the increase in the number of deaths and the occupation of the hospitals beds reaching their maximum occupancy, I would like to conduct a study where it points out areas of lower risk and high options for health treatment in general. 

#### The aim of the project is to apply the skills learned in the Coursera course to find the safest neighborhood in Rio de Janeiro, surrounded by hospitals, drugstores, clinics and so on. This will be determined by analyzing the number of cases of covid, deaths and the profile of the neighborhoods population, clustering neighborhoods using k-means and exploring on the map the top common healthy venues in the safest neighborhoods. 

####This exercise may also be of interest to anyone who is facing this pandemic and is concerned about the health issue.


---


---




## Data

#### The Rio Covid Panel is an initiative of the city of Rio de Janeiro. It's an open source and available for download directly from the Data Rio Website [website](https://www.data.rio/app/painel-rio-covid-19). It contained the features about COVID cases: neighborhood of residence, case evolution, age group and sex. 

#### I also obtained a list of Rio de Janeiro’s districts and neighborhoods by web scraping a [Wikipedia page](https://pt.wikipedia.org/wiki/Lista_de_bairros_da_cidade_do_Rio_de_Janeiro) using BeautifulSoup. However, this dataset lacked the geographical coordinates. So, I used geocoder to obtain the latitude and longitude coordinates for each Rio de Janeiro neighborhood.

#### I've got the number of habitants in each neighborhoood in Rio de Janeiro from from the Data Rio Website [website](https://www.data.rio/search?groupIds=0f4009068ec74e17b25eb3e70891b95f&sort=-modified). It's ans open source information and available to download for free.

#### To get venue information in each neighborhood, I called the [Foursquare API](https://foursquare.com/developers/apps). This gave me a dataset containing the venue name, latitude and longitude coordinates of the venue location, and the venue category.